# Homework
-------
1. Complete the codes just like what I did in the video.
- Add Logistic regression model and run the codes.  You should analyze bias just like what we have already done with other 4 models.
- Use different dataset to do the analysis.  Ideal dataset should be a binary classification.  Kaggle.com is a good website to look for the dataset. (selected dataset: [Pumpkin Seeds Dataset](https://www.kaggle.com/datasets/muratkokludataset/pumpkin-seeds-dataset))

In [71]:
import pandas as pd
import numpy as np
from scipy.io import arff
from os.path import join
from os import system, getcwd

## 1. Dataset subroutines

In [72]:
def get_bcw() :
    path = join(getcwd().rstrip('src'), 'data/Pumpkin_Seeds_Dataset.arff').replace('\\', '/')
    bcw = arff.loadarff(path)
    bcw = pd.DataFrame(bcw[0])
    
    bcw['Class'] = bcw['Class'].str.decode("utf-8")
    bcw['Class'] = bcw['Class'].map(lambda x: 0 if x == "CERCEVELIK" else 1)

    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X = ss.fit_transform(bcw)
    
    
    y = bcw['Class'].values
    return X, y

# 2. get dataset

In [73]:
X, y = get_bcw()   # X: feature vector, y label

In [74]:
X_orig = X
y_orig = y

bcw = pd.DataFrame(X)
bcw['Class'] = y

bcw.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,Class
0,-1.784703,-2.215755,-2.320224,-0.238536,-1.793218,-1.921174,-2.729935,0.202812,0.855406,1.873763,-1.775063,2.197280,-0.960769,0
1,-0.294780,-0.568804,-0.700916,0.362089,-0.307244,-0.259343,-0.739163,0.603626,0.359523,0.938377,-0.824864,0.840230,-0.960769,0
2,-0.661350,-0.432940,-0.369395,-0.633215,-0.642749,-0.645380,0.310476,-1.085518,0.768380,-0.431615,0.074059,-0.211483,-0.960769,0
3,-1.039412,-1.265424,-1.334616,-0.140076,-1.045689,-1.057939,-1.075756,0.202812,0.761812,1.020648,-1.035351,1.098447,-0.960769,0
4,-1.065105,-1.209626,-1.293272,-0.229276,-1.045762,-1.086548,-0.934033,-1.285925,-0.295634,0.755950,-0.950840,0.972166,-0.960769,0


In [75]:
bcw.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'Class'], dtype='object')

In [76]:
cols = bcw.columns
['Class'] + list(cols[:-1])

['Class', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [77]:
# try to move 'malignant' column to the front.
cols = bcw.columns
# print(['malignant'] + list(cols[:-1]))
bcw = bcw[['Class'] + list(cols[:-1])]
bcw.head()

,Class,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,-1.784703,-2.215755,-2.320224,-0.238536,-1.793218,-1.921174,-2.729935,0.202812,0.855406,1.873763,-1.775063,2.197280,-0.960769
1,0,-0.294780,-0.568804,-0.700916,0.362089,-0.307244,-0.259343,-0.739163,0.603626,0.359523,0.938377,-0.824864,0.840230,-0.960769
2,0,-0.661350,-0.432940,-0.369395,-0.633215,-0.642749,-0.645380,0.310476,-1.085518,0.768380,-0.431615,0.074059,-0.211483,-0.960769
3,0,-1.039412,-1.265424,-1.334616,-0.140076,-1.045689,-1.057939,-1.075756,0.202812,0.761812,1.020648,-1.035351,1.098447,-0.960769
4,0,-1.065105,-1.209626,-1.293272,-0.229276,-1.045762,-1.086548,-0.934033,-1.285925,-0.295634,0.755950,-0.950840,0.972166,-0.960769


In [78]:
def GetImbalancedLabelData(DF_orig, label_ratio) :
    import copy as copy
    from sklearn.model_selection import train_test_split
    DF = copy.copy(DF_orig)
    DF1 = DF[DF.Class == 1]
    DF0 = DF[DF.Class == 0]
    
    Test_Size= 0.25
    max_train0_size = min(DF0.shape[0], DF1.shape[0])
    TestSizeFrom0 =1 -  min(max_train0_size, DF0.shape[0] * (1- Test_Size) ) / DF0.shape[0]
    
    
    DF0_train, DF0_test = train_test_split(DF0, test_size = TestSizeFrom0, random_state=2018)
    
    DF1_train, DF1_test = train_test_split(DF1, test_size =  (1 - label_ratio), random_state=2018)
    
    DF_train = pd.concat((DF0_train, DF1_train) )
    DF_test = pd.concat((DF0_test, DF1_test) )
    
    y_train = DF_train.Class.values
    X_train = DF_train.drop(columns=['Class']).values
    y_test = DF_test.Class.values
    X_test = DF_test.drop(columns=['Class']).values

    return X_train, X_test, y_train, y_test

In [79]:
X_train, X_test, y_train, y_test = GetImbalancedLabelData(bcw, 0.1)

# 3. Split Data into Training and Testing
--------
Choose 3.1  and 3.2 to get the type of split data you want to train.

# 3.1 Original Data Split

In [80]:
from sklearn.model_selection import train_test_split

print(X.shape)
print(y.shape)
print(type(X))
print(type(y))
# numpy array does not have column names.  
# We can only use the slice [row_start:row_end:step_row, col_start:col_end:step_col]
# to select rows and cols we want
# be aware that row_end and col_end are not inclusive
# the format of the output will be discussed in another lecture.
print(X[0:6:2])
print(y[0:40:4])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=2018) 


(2500, 13)
(2500,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-1.78470346 -2.21575484 -2.32022415 -0.23853605 -1.79321817 -1.92117425
  -2.72993503  0.20281179  0.85540604  1.8737626  -1.77506344  2.19727996
  -0.96076892]
 [-0.66135033 -0.43294002 -0.36939513 -0.63321531 -0.64274879 -0.64537979
   0.31047626 -1.08551833  0.76838021 -0.43161453  0.07405883 -0.21148339
  -0.96076892]
 [-1.06510454 -1.20962642 -1.29327246 -0.22927559 -1.04576187 -1.08654848
  -0.93403295 -1.28592524 -0.29563362  0.75594979 -0.95083981  0.97216558
  -0.96076892]]
[0 0 0 0 0 0 0 0 0 0]


# 3.2 Data Split to Get Imbalanced Labels in Training Data

In [81]:
def GetImbalancedLabelData(DF_orig, label_ratio) :
    import copy as copy
    from sklearn.model_selection import train_test_split
    DF = copy.copy(DF_orig)
    DF1 = DF[DF.Class == 1]
    DF0 = DF[DF.Class == 0]
    
    Test_Size= 0.25  # intended test size.  The actual size depend on how the train data is arranged.
    # Simple implementation : 
    #      split both DF0 with Test_Size and DF1 with label_ratio 
    #      This method can not produce the right label_ratio for training set. 
#     TestSizeFrom0 =  Test_Size
    # instead , the following 2 statement, can split the training data in the precise label_ratio.
    max_train0_size = min(DF0.shape[0], DF1.shape[0])  
    TestSizeFrom0 = 1 -  min(max_train0_size, DF0.shape[0] * (1- Test_Size) ) / DF0.shape[0]
    
    
    DF0_train, DF0_test = train_test_split(DF0, test_size = TestSizeFrom0, random_state=2018)
    DF1_train, DF1_test = train_test_split(DF1, test_size =  (1 - label_ratio), random_state=2018)
    
    DF_train = pd.concat((DF0_train, DF1_train) )
    DF_test = pd.concat((DF0_test, DF1_test) )
    
    y_train = DF_train.Class.values
    X_train = DF_train.drop(columns=[ 'Class']).values
    y_test = DF_test.Class.values
    X_test = DF_test.drop(columns=[ 'Class']).values

    return X_train, X_test, y_train, y_test

In [82]:
X_train, X_test, y_train, y_test = GetImbalancedLabelData(bcw, 0.5)
print(y_train.sum(), y_train.shape,  y_test.sum(), y_test.shape,)
print(y_train.sum()/( y_train.shape[0] - y_train.sum() )  )

600 (1575,) 600 (925,)
0.6153846153846154


# 4. Build ML/DL model (model specific)

In [83]:
def evaluate(pred, expect) :
    ans = pred - expect
    error_sum = ans.sum()
    n_errors = abs(ans).sum()
    accuracy = 1 - n_errors / expect.shape[0]
    
    return round(accuracy, 3) , n_errors, error_sum

In [84]:
def RunAllModels(bcw, Label_Ratio, default=True):
    X_train, X_test, y_train, y_test = GetImbalancedLabelData(bcw, 0.5)
    Result = []
    
    from sklearn.tree import DecisionTreeClassifier
    if default:
        model = DecisionTreeClassifier()
    else:
        model = DecisionTreeClassifier(class_weight='balanced')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    Result.append(['DT', default, round(Label_Ratio, 2)] + list(evaluate(pred, y_test)))

    from sklearn.ensemble import RandomForestClassifier
    if default:
        model = RandomForestClassifier(n_estimators=100)
    else:
        model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    Result.append(['RF', default, round(Label_Ratio, 2)] + list(evaluate(pred, y_test)))

    from sklearn.neighbors import KNeighborsClassifier
    if default:
        model = KNeighborsClassifier()
    else:
        model = KNeighborsClassifier(weights='distance')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    Result.append(['KNN', default, round(Label_Ratio, 2)] + list(evaluate(pred, y_test)))

    from sklearn.svm import SVC
    if default:
        model = SVC()
    else:
        # # model = SVC(gamma='scale')
        model = SVC(gamma='scale', class_weight='balanced')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    Result.append(['SVC', default, round(Label_Ratio, 2)] + list(evaluate(pred, y_test)))
    
    from sklearn.linear_model import LogisticRegression
    if default:
        model = LogisticRegression()
    else:
        model = LogisticRegression(class_weight='balanced')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    Result.append(['LR', default, round(Label_Ratio, 2)] + list(evaluate(pred, y_test)))
    
    return Result

In [85]:
Default = []
for i in range(10, 100):
    labelratio = 0.01 * i
    Default += RunAllModels(bcw, labelratio)
Default

[['DT', True, 0.1, 1.0, 0, 0],
 ['RF', True, 0.1, 1.0, 0, 0],
 ['KNN', True, 0.1, 0.995, 5, -3],
 ['SVC', True, 0.1, 0.998, 2, 2],
 ['LR', True, 0.1, 1.0, 0, 0],
 ['DT', True, 0.11, 1.0, 0, 0],
 ['RF', True, 0.11, 1.0, 0, 0],
 ['KNN', True, 0.11, 0.995, 5, -3],
 ['SVC', True, 0.11, 0.998, 2, 2],
 ['LR', True, 0.11, 1.0, 0, 0],
 ['DT', True, 0.12, 1.0, 0, 0],
 ['RF', True, 0.12, 1.0, 0, 0],
 ['KNN', True, 0.12, 0.995, 5, -3],
 ['SVC', True, 0.12, 0.998, 2, 2],
 ['LR', True, 0.12, 1.0, 0, 0],
 ['DT', True, 0.13, 1.0, 0, 0],
 ['RF', True, 0.13, 1.0, 0, 0],
 ['KNN', True, 0.13, 0.995, 5, -3],
 ['SVC', True, 0.13, 0.998, 2, 2],
 ['LR', True, 0.13, 1.0, 0, 0],
 ['DT', True, 0.14, 1.0, 0, 0],
 ['RF', True, 0.14, 1.0, 0, 0],
 ['KNN', True, 0.14, 0.995, 5, -3],
 ['SVC', True, 0.14, 0.998, 2, 2],
 ['LR', True, 0.14, 1.0, 0, 0],
 ['DT', True, 0.15, 1.0, 0, 0],
 ['RF', True, 0.15, 1.0, 0, 0],
 ['KNN', True, 0.15, 0.995, 5, -3],
 ['SVC', True, 0.15, 0.998, 2, 2],
 ['LR', True, 0.15, 1.0, 0, 0],
 ['

In [86]:
Weight = []
for i in range(10, 100):
    labelratio = 0.01 * i
    Weight += RunAllModels(bcw, labelratio, True)
Weight

[['DT', True, 0.1, 1.0, 0, 0],
 ['RF', True, 0.1, 1.0, 0, 0],
 ['KNN', True, 0.1, 0.995, 5, -3],
 ['SVC', True, 0.1, 0.998, 2, 2],
 ['LR', True, 0.1, 1.0, 0, 0],
 ['DT', True, 0.11, 1.0, 0, 0],
 ['RF', True, 0.11, 1.0, 0, 0],
 ['KNN', True, 0.11, 0.995, 5, -3],
 ['SVC', True, 0.11, 0.998, 2, 2],
 ['LR', True, 0.11, 1.0, 0, 0],
 ['DT', True, 0.12, 1.0, 0, 0],
 ['RF', True, 0.12, 1.0, 0, 0],
 ['KNN', True, 0.12, 0.995, 5, -3],
 ['SVC', True, 0.12, 0.998, 2, 2],
 ['LR', True, 0.12, 1.0, 0, 0],
 ['DT', True, 0.13, 1.0, 0, 0],
 ['RF', True, 0.13, 1.0, 0, 0],
 ['KNN', True, 0.13, 0.995, 5, -3],
 ['SVC', True, 0.13, 0.998, 2, 2],
 ['LR', True, 0.13, 1.0, 0, 0],
 ['DT', True, 0.14, 1.0, 0, 0],
 ['RF', True, 0.14, 1.0, 0, 0],
 ['KNN', True, 0.14, 0.995, 5, -3],
 ['SVC', True, 0.14, 0.998, 2, 2],
 ['LR', True, 0.14, 1.0, 0, 0],
 ['DT', True, 0.15, 1.0, 0, 0],
 ['RF', True, 0.15, 1.0, 0, 0],
 ['KNN', True, 0.15, 0.995, 5, -3],
 ['SVC', True, 0.15, 0.998, 2, 2],
 ['LR', True, 0.15, 1.0, 0, 0],
 ['

# 5. Get Some important statistics for variosu labelratio and models with groupby and other statistics

In [87]:
DefaultDF = pd.DataFrame(Default, columns = ['Model', 'Weight', 'LabelRatio', 'Accuracy', 'N_errors', 'Bias'])
WeightDF = pd.DataFrame(Weight, columns = ['Model', 'Weight', 'LabelRatio', 'Accuracy', 'N_errors', 'Bias'])

In [88]:
print('DefaultDF: ' + str(DefaultDF.Bias.sum()))
print('WeightDF:  ' + str(WeightDF.Bias.sum()))

DefaultDF: -90
WeightDF:  -90


In [89]:
DefaultDF.describe()

,LabelRatio,Accuracy,N_errors,Bias
count,450.000000,450.000000,450.000000,450.000000
mean,0.545000,0.998600,1.400000,-0.200000
std,0.260081,0.001962,1.961773,1.601781
min,0.100000,0.995000,0.000000,-3.000000
25%,0.320000,0.998000,0.000000,0.000000
50%,0.545000,1.000000,0.000000,0.000000
75%,0.770000,1.000000,2.000000,0.000000
max,0.990000,1.000000,5.000000,2.000000


In [90]:
WeightDF.describe()

,LabelRatio,Accuracy,N_errors,Bias
count,450.000000,450.000000,450.000000,450.000000
mean,0.545000,0.998600,1.400000,-0.200000
std,0.260081,0.001962,1.961773,1.601781
min,0.100000,0.995000,0.000000,-3.000000
25%,0.320000,0.998000,0.000000,0.000000
50%,0.545000,1.000000,0.000000,0.000000
75%,0.770000,1.000000,2.000000,0.000000
max,0.990000,1.000000,5.000000,2.000000


In [91]:
DefaultDF[DefaultDF.LabelRatio > 0.7].describe()

,LabelRatio,Accuracy,N_errors,Bias
count,145.000000,145.000000,145.000000,145.000000
mean,0.850000,0.998600,1.400000,-0.200000
std,0.083956,0.001966,1.966384,1.605546
min,0.710000,0.995000,0.000000,-3.000000
25%,0.780000,0.998000,0.000000,0.000000
50%,0.850000,1.000000,0.000000,0.000000
75%,0.920000,1.000000,2.000000,0.000000
max,0.990000,1.000000,5.000000,2.000000


In [92]:
WeightDF[WeightDF.LabelRatio > 0.7].describe()

,LabelRatio,Accuracy,N_errors,Bias
count,145.000000,145.000000,145.000000,145.000000
mean,0.850000,0.998600,1.400000,-0.200000
std,0.083956,0.001966,1.966384,1.605546
min,0.710000,0.995000,0.000000,-3.000000
25%,0.780000,0.998000,0.000000,0.000000
50%,0.850000,1.000000,0.000000,0.000000
75%,0.920000,1.000000,2.000000,0.000000
max,0.990000,1.000000,5.000000,2.000000


In [93]:
DefaultDF[(DefaultDF.LabelRatio < 0.7) & (DefaultDF.LabelRatio > 0.2)].describe()

,LabelRatio,Accuracy,N_errors,Bias
count,245.000000,245.000000,245.000000,245.000000
mean,0.450000,0.998600,1.400000,-0.200000
std,0.141711,0.001964,1.963603,1.603275
min,0.210000,0.995000,0.000000,-3.000000
25%,0.330000,0.998000,0.000000,0.000000
50%,0.450000,1.000000,0.000000,0.000000
75%,0.570000,1.000000,2.000000,0.000000
max,0.690000,1.000000,5.000000,2.000000


In [94]:
WeightDF[(WeightDF.LabelRatio < 0.7) & (WeightDF.LabelRatio > 0.2)].describe()

,LabelRatio,Accuracy,N_errors,Bias
count,245.000000,245.000000,245.000000,245.000000
mean,0.450000,0.998600,1.400000,-0.200000
std,0.141711,0.001964,1.963603,1.603275
min,0.210000,0.995000,0.000000,-3.000000
25%,0.330000,0.998000,0.000000,0.000000
50%,0.450000,1.000000,0.000000,0.000000
75%,0.570000,1.000000,2.000000,0.000000
max,0.690000,1.000000,5.000000,2.000000


In [95]:
DefaultDF.groupby('Model').sum()

,Weight,LabelRatio,Accuracy,N_errors,Bias
Model,,,,,
DT,90,49.05,90.00,0,0
KNN,90,49.05,89.55,450,-270
LR,90,49.05,90.00,0,0
RF,90,49.05,90.00,0,0
SVC,90,49.05,89.82,180,180


In [96]:
WeightDF.groupby('Model').sum()

,Weight,LabelRatio,Accuracy,N_errors,Bias
Model,,,,,
DT,90,49.05,90.00,0,0
KNN,90,49.05,89.55,450,-270
LR,90,49.05,90.00,0,0
RF,90,49.05,90.00,0,0
SVC,90,49.05,89.82,180,180


In [97]:
DefaultDF[DefaultDF.LabelRatio > 0.7].groupby('Model').sum()

,Weight,LabelRatio,Accuracy,N_errors,Bias
Model,,,,,
DT,29,24.65,29.000,0,0
KNN,29,24.65,28.855,145,-87
LR,29,24.65,29.000,0,0
RF,29,24.65,29.000,0,0
SVC,29,24.65,28.942,58,58


In [98]:
WeightDF[WeightDF.LabelRatio > 0.7].groupby('Model').sum()

,Weight,LabelRatio,Accuracy,N_errors,Bias
Model,,,,,
DT,29,24.65,29.000,0,0
KNN,29,24.65,28.855,145,-87
LR,29,24.65,29.000,0,0
RF,29,24.65,29.000,0,0
SVC,29,24.65,28.942,58,58
